In [2]:
from pprint import pprint
import pandas as pd
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import mlflow
import mlflow.xgboost


from mlflow import log_metric, log_param, log_artifacts

from sklearn.metrics import make_scorer, f1_score, recall_score,precision_score, get_scorer_names, accuracy_score,confusion_matrix, roc_curve
from mlflow import log_metric, log_param, log_artifacts

import seaborn as sn
import matplotlib.pyplot as plt

import joblib

from sklearn.ensemble import IsolationForest
from sklearn.metrics import make_scorer, f1_score
from sklearn import model_selection
from sklearn.datasets import make_classification


# Read data

In [3]:
fraudes_df = pd.read_csv('datos_regiones_especiales_201906-202205.csv').drop(['grupo','NombreRegion'],axis=1)#.drop(['LOCALIDAD','MUNICIPIO'], axis = 1)


#fraudes_df = pd.merge(fraudes_df, actividad_econ_df,how='inner', on = ['nis_rad']).drop('Unnamed: 0', axis=1)
fraudes_df.head()

,LlavePuntoConsumo,202106,202107,202108,202109,202110,202111,202112,202201,202202,202203,202204,202205,AlturaPuntoConsumo,NombreDepartamentoGeografia,NombreMunicipioGeografia,seccion,DescripcionAnomalia,anomalias_anuales,fraude
0,1002467,122.72,111.53,113.11,97.14,102.40,95.17,121.67,125.47,133.44,114.80,124.71,123.69,1784.0,CHIMALTENANGO,CHIMALTENANGO,Residenciales,NO REGSITRADO,0,1
1,1017184,0.00,3.15,10.46,114.57,1.01,1.90,1.05,0.95,0.00,0.00,0.00,0.00,1779.0,CHIMALTENANGO,SAN MARTIN JILOTEPEQUE,Residenciales,Presunto Fraude,0,1
2,2405770,111.18,147.18,161.90,236.47,344.72,323.79,90.20,81.74,122.65,171.71,270.71,245.36,2530.0,SOLOLA,SANTA LUCIA UTATLAN,Residenciales,NO REGSITRADO,0,1
3,2442128,58.87,62.93,94.19,294.35,227.11,207.21,205.57,212.92,238.58,219.00,255.50,208.99,2033.0,QUICHE,SANTA CRUZ DEL QUICHE,Residenciales,NO REGSITRADO,0,1
4,3161851,25.51,13.64,2.94,11.41,11.54,13.31,23.07,27.57,39.25,37.51,42.58,41.21,921.0,SANTA ROSA,CUILAPA,Residenciales,Tapa rota/falta,0,1


## Data Processing

### Handle categoricals

In [4]:
ohe_features = fraudes_df


In [5]:
ohe_NombreDepartamentoGeografia = OneHotEncoder()
ohe_NombreMunicipioGeografia = OneHotEncoder()
ohe_seccion = OneHotEncoder()
ohe_DescripcionAnomalia = OneHotEncoder()

transformed_NombreDepartamentoGeografia = ohe_NombreDepartamentoGeografia.fit_transform(ohe_features[['NombreDepartamentoGeografia']])
ohe_features[ohe_NombreDepartamentoGeografia.categories_[0]] = transformed_NombreDepartamentoGeografia.toarray()

transformed_NombreMunicipioGeografia = ohe_NombreMunicipioGeografia.fit_transform(ohe_features[['NombreMunicipioGeografia']])
ohe_features[ohe_NombreMunicipioGeografia.categories_[0]] = transformed_NombreMunicipioGeografia.toarray()

transformed_seccion = ohe_seccion.fit_transform(ohe_features[['seccion']])
ohe_features[ohe_seccion.categories_[0]] = transformed_seccion.toarray()

transformed_DescripcionAnomalia = ohe_DescripcionAnomalia.fit_transform(ohe_features[['DescripcionAnomalia']])
ohe_features[ohe_DescripcionAnomalia.categories_[0]] = transformed_DescripcionAnomalia.toarray()

ohe_features = ohe_features.drop(['NombreDepartamentoGeografia','NombreMunicipioGeografia','seccion','DescripcionAnomalia'], axis = 1)

/tmp/ipykernel_1502/498564259.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ohe_features[ohe_NombreMunicipioGeografia.categories_[0]] = transformed_NombreMunicipioGeografia.toarray()
/tmp/ipykernel_1502/498564259.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ohe_features[ohe_NombreMunicipioGeografia.categories_[0]] = transformed_NombreMunicipioGeografia.toarray()
/tmp/ipykernel_1502/498564259.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times,

In [6]:
y = ohe_features['fraude']

X = ohe_features



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.drop(['fraude','LlavePuntoConsumo'], axis = 1)

nis_x_test = X_test['LlavePuntoConsumo']

X_test = X_test.drop(['fraude','LlavePuntoConsumo'], axis = 1)
                      
X_test_full = X_test

In [8]:
mlflow.set_experiment("Energuate-Regiones-especiales-supervisado")

def yield_artifacts(run_id, path=None):
    """Yield all artifacts in the specified run"""
    client = mlflow.tracking.MlflowClient()
    for item in client.list_artifacts(run_id, path):
        if item.is_dir:
            yield from yield_artifacts(run_id, item.path)
        else:
            yield item.path


def fetch_logged_data(run_id):
    """Fetch params, metrics, tags, and artifacts in the specified run"""
    client = mlflow.tracking.MlflowClient()
    data = client.get_run(run_id).data
    # Exclude system tags: https://www.mlflow.org/docs/latest/tracking.html#system-tags
    tags = {k: v for k, v in data.tags.items() if not k.startswith("mlflow.")}
    artifacts = list(yield_artifacts(run_id))
    return {
        "params": data.params,
        "metrics": data.metrics,
        "tags": tags,
        "artifacts": artifacts,
    }



def main():
    
    mlflow.xgboost.autolog()

    clf = xgb.XGBClassifier()#(n_estimators=60, reg_lambda=10, gamma=70, max_depth=30)
    clf.fit(X_train.values, y_train.values, eval_set=[(X_test.values, y_test.values)])
    
    run_id = mlflow.last_active_run().info.run_id
    print("Logged data and model in run {}".format(run_id))

    # show logged data
    for key, data in fetch_logged_data(run_id).items():
        print("\n---------- logged {} ----------".format(key))
        pprint(data)

        
        
    with mlflow.start_run(run_id=run_id) as run:
        print('---TEST_METRICS----')
        
        
        y_pred = clf.predict(X_test.values)
        y_true = y_test.values
        print(y_pred)
        print(y_true)
        f1 = f1_score(y_true, y_pred, average='binary')
        log_metric("test_f1_score", f1)
        print('test_score_f1 '+str(f1))
        recall = recall_score(y_true, y_pred, average='binary')
        log_metric("test_recall_score", recall)
        print('test_recall_f1 '+str(recall))
        accuracy = accuracy_score(y_true, y_pred)
        log_metric("test_accuracy_score", accuracy)
        print('test_score_accuracy '+str(accuracy))
        precision = precision_score(y_true, y_pred)
        log_metric("test_precision_score", precision)
        print('test_score_precision '+str(precision))
        log_param("estimator", "XGBoostC")
        
        

if __name__ == "__main__":
    main()

2022/10/26 08:32:25 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '344055f70e1a4a28a51b848c158e4809', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[0]	validation_0-logloss:0.60719
[1]	validation_0-logloss:0.55770
[2]	validation_0-logloss:0.52599
[3]	validation_0-logloss:0.50408
[4]	validation_0-logloss:0.49142
[5]	validation_0-logloss:0.48136
[6]	validation_0-logloss:0.47077
[7]	validation_0-logloss:0.46635
[8]	validation_0-logloss:0.45654
[9]	validation_0-logloss:0.45320
[10]	validation_0-logloss:0.44890
[11]	validation_0-logloss:0.44771
[12]	validation_0-logloss:0.44589
[13]	validation_0-logloss:0.44425
[14]	validation_0-logloss:0.44241
[15]	validation_0-logloss:0.44016
[16]	validation_0-logloss:0.43879
[17]	validation_0-logloss:0.43746
[18]	validation_0-logloss:0.43707
[19]	validation_0-logloss:0.43572
[20]	validation_0-logloss:0.43452
[21]	validation_0-logloss:0.43438
[22]	validation_0-logloss:0.43358
[23]	validation_0-logloss:0.43189
[24]	validation_0-logloss:0.43212
[25]	validation_0-logloss:0.43169
[26]	validation_0-logloss:0.43138
[27]	validation_0-logloss:0.42937
[28]	validation_0-logloss:0.42925
[29]	validation_0-loglos

In [ ]:
def main():
    #480 modelos
    mlflow.set_experiment("XGBoost")
    n_estimators = [10, 100,200,300]
    max_depth = [10,20,30,50,100]
    reg_lambda=[1,5,10,15,25,50]
    gamma=[1,5,10,20]
    iterables = [n_estimators, max_depth, reg_lambda, gamma]

    iris = datasets.load_iris()
    # prepare example dataset
    X, y = load_iris(return_X_y=True, as_frame=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    
    
    for t in itertools.product(*iterables):
        print(t)
        # enable auto logging
        # this includes xgboost.sklearn estimators
        mlflow.xgboost.autolog()
        xgb_model = xgb.XGBRegressor(n_jobs=-1,n_estimators=t[0], max_depth=t[1], reg_lambda=t[2], gamma=t[3])
        xgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)])
        y_pred = xgb_model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        run_id = mlflow.last_active_run().info.run_id
        print("Logged data and model in run {}".format(run_id))
        # show logged data
        for key, data in fetch_logged_data(run_id).items():
            print("\n---------- logged {} ----------".format(key))
            pprint(data)



if __name__ == "__main__":
    main()

In [ ]:
import itertools

for t in itertools.product(*iterables):
    print(t)
